# Showcase of Bybit Future - Future calc
The Bybitmaster has custom methods that I coded, but can also make normal request to the Bybit API with the session attribute.

In [1]:
# Inside of the module, we use absolute imports
# Because this context is different, we need to add inside the PYTHONPATH the path of the module
import sys
sys.path.append('Bybit')

from Bybit.greekMaster import GreekMaster
import Bybit.utils


# Create a Bybit master
master = GreekMaster(demo=False, verbose=1)

2024-11-30 17:24:36,395 - Bybit.greekMaster - INFO - GreekMaster initialized


In [3]:
# Get all futures BTC gaps
DataWrangler = master.all_gaps_pd(inverse=False, perpetual=True, pretty=True, applyFees=True)
DataWrangler

,Buy,Sell,Gap,Coeff,APR,DaysLeft,CumVolume
0,BTCPERP,BTCUSDT,$ -68.50,0.00 %,-0.00 %,-20056,4.95B
1,BTCPERP,BTC-06DEC24,$ 293.10,0.08 %,4.53 %,6,35.17M
2,BTCPERP,BTC-13DEC24,$ 655.60,0.46 %,12.23 %,13,34.81M
3,BTCPERP,BTC-20DEC24,$ 1003.10,0.81 %,14.42 %,20,34.44M
4,BTCPERP,BTC-27DEC24,$ 1336.10,1.16 %,15.30 %,27,35.39M
5,BTCPERP,BTC-31JAN25,$ 2867.10,2.73 %,15.94 %,62,34.77M
6,BTCPERP,BTC-28MAR25,$ 4858.10,4.79 %,14.73 %,118,34.75M
7,BTCPERP,BTC-27JUN25,$ 8109.10,8.14 %,14.17 %,209,34.85M
8,BTCPERP,BTC-26SEP25,$ 11357.60,11.49 %,13.95 %,300,34.90M
9,BTCUSDT,BTC-06DEC24,$ 361.60,0.15 %,8.45 %,6,4.92B


In [ ]:
master.client.fetcher.get_USDC_BTC()
# BTCUSDT	BTC-29NOV24

await master.client.fetcher.exit_both_position('BTCUSDT', 'BTC-20DEC24', 0.010, 0.014)


## Tester
Just to run some tests

In [ ]:
# 1 - Simple enter position logic (given a future and a perpetual)
# 2 - Simple exit position logic (given a future and a perpetual). Can also exit when reaching a satisfying unexpected gap
# 3 - Logic to activate both operations (given a future and a perpetual). Enter again when reaching a satisfying unexpected gap
# 4 - Logic to find the best future and perpetual to trade
# 5 - Logic to keep the loop running
# 6 - Monitoring features
# 7 - Use systemctl to launch 24/7
# We use sockets, once we selected the best local future, compared to their average gap.
# Shared state to store the latest updates

## Fetching time :)

Getting 15, 5 and 1 minute candles for everything since 01/01/2024

In [ ]:
# Future contracts
allContracts = master.client.fetcher.get_futureNames()

for contract in allContracts:
    master.client.fetcher.get_history_pd(contract, interval="15", dest="store", dateLimit="2024-01-01 00:00")
    master.client.fetcher.get_history_pd(contract, interval="5", dest="store", dateLimit="2024-01-01 00:00")
    master.client.fetcher.get_history_pd(contract, interval="1", dest="store", dateLimit="2024-01-01 00:00")

# spot
# client.fetcher.get_history_pd("BTCUSDC", interval="15", dateLimit="2024-01-01 00:00", category="spot", dest="store")
master.client.fetcher.get_history_pd("BTCUSDT", interval="15", dateLimit="2024-01-01 00:00", category="spot", dest="store")
master.client.fetcher.get_history_pd("BTCUSDT", interval="5", dateLimit="2024-01-01 00:00", category="spot", dest="store")
master.client.fetcher.get_history_pd("BTCUSDT", interval="1", dateLimit="2024-01-01 00:00", category="spot", dest="store")

# Perpetual contracts
master.client.fetcher.get_history_pd("BTCUSDT", interval="15", dateLimit="2024-01-01 00:00", dest="store")
master.client.fetcher.get_history_pd("BTCUSDT", interval="5", dateLimit="2024-01-01 00:00", dest="store")
master.client.fetcher.get_history_pd("BTCUSDT", interval="1", dateLimit="2024-01-01 00:00", dest="store")


## Plot testing

Part to test the plot of the data.


In [ ]:
# Yes

## Long Spot x Short Future

Spot USDC is 0% fees and Future is 0.0550% taker fees (0% with contract delivery).
Problem is that spot cannot be used with leverage. Could borrow but that would be awkward. (tried it)

In [ ]:
Bybit.utils.load_klines_parquet("store/BTCUSDT_1.parquet", pretty=True)

Bybit.utils.plot_compare("store/BTCUSDT_1.parquet", "store/BTC-20DEC24_1.parquet")

In [ ]:
# Long position, then short position
Bybit.utils.plot_compare("store/BTCUSDT_1.parquet", "store/BTC-27DEC24_1.parquet", upperlimit="2024-04-08 00:00")

## Long Perpetual x Short Future

Perpetual USDT is 0.0550% taker fees and Future is 0.0550% taker fees (0% with contract delivery).
Yes we have fees, but wen can use leverage on both sides.

We use Perpetual USDT because there is more liquidity.

In [ ]:
Bybit.utils.load_klines_parquet("store/BTCUSDT_15.parquet", pretty=True)

In [ ]:
Bybit.utils.plot_compare("store/BTCUSDT_15.parquet", "store/BTC-29NOV24_15.parquet")

## Long Spot x Short Inverse Future

What if I buy spot and collateralize it to short inverse future?
We effectively capture the difference.

In [ ]:
# Keep going when get_history_pd file names are changed

## OKX 20x Leverage test

I launched a test with OKX with a short on BTCUSDT-27JUN25 and long on BTCUSDT-29NOV24.

Following section will track the the two contracts' data

In [ ]:
Bybit.utils.plot_compare("store/BTC-29NOV24_15.parquet", "store/BTC-27JUN25_15.parquet", dateLimit="2024-05-06")